In [ ]:
import numpy as np

In [ ]:
DIM_PUZZLE = 5
#goal = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]])
#goal = np.array([[1,2,3],[4,5,6],[7,8,0]])
goal = np.array([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15], [16, 17, 18, 19 ,20], [21, 22, 23, 24, 0]])


In [ ]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
action = namedtuple('Action', ['pos1', 'pos2'])
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < DIM_PUZZLE - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < DIM_PUZZLE - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

RANDOMIZE_STEPS = 100_000

state = np.array([i for i in range(1, DIM_PUZZLE**2)] + [0]).reshape((DIM_PUZZLE, DIM_PUZZLE))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

In [ ]:
def linear_conflicts(state, goal):
   
    n = DIM_PUZZLE
    conflicts = 0
    
    # Map the elements in the goal state to their positions
    goal_positions = {}
    for r in range(n):
        for c in range(n):
            goal_positions[goal[r][c]] = (r, c)

    # Verify conflicts row by row
    for row in range(n):
        conflicts += count_linear_conflicts(state[row], goal_positions, row, is_row=True)
    
    # Verify conflicts column by column
    for col in range(n):
        column = [state[row][col] for row in range(n)]  # Extract the column
        conflicts += count_linear_conflicts(column, goal_positions, col, is_row=False)

    return conflicts


def count_linear_conflicts(line, goal_positions, index, is_row=True):
    """
    - line: list of elements of current row or current column
    - goal_positions: dict mapped from goal state
    - index: index of current row or column
    - is_row: True if we are analyzing a row, false otherwise
    
    """
    conflicts = 0
    n = len(line)
    for i in range(n):
        if line[i] == 0:  # Ignore the empty element
            continue
        for j in range(i + 1, n):
            if line[j] == 0:  # Ignore the empty element
                continue
            # get the goal positions of both elements
            goal_i = goal_positions[line[i]]
            goal_j = goal_positions[line[j]]

            if is_row:
                # they both have to be in the same row
                if goal_i[0] == index and goal_j[0] == index and goal_i[1] > goal_j[1]:
                    conflicts += 1
            else:
                # they both have to be in the same column
                if goal_i[1] == index and goal_j[1] == index and goal_i[0] > goal_j[0]:
                    conflicts += 1

    return conflicts

In [ ]:
man = 0 #manhattan distance

def h(state, goal):
    man = 0 # manhattan distance
    for i in range(DIM_PUZZLE):
        for j in range(DIM_PUZZLE):
            man += abs(state[i][j]-goal[i][j])
            
    
    conflicts = linear_conflicts(state, goal)


    man += 2*conflicts
    return man



In [ ]:
from queue import PriorityQueue

class Node:
    def __init__(self, state, parent=None, action=None, heuristic=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.heuristic = heuristic

    def __lt__(self, other): # choose the node with the lowest heuristic value
        return self.heuristic < other.heuristic

def greedy_best_first_search(start, goal, heuristic_func):

    open_list = PriorityQueue() # nodes to be explored
    closed_list = set() # nodes already explored
    start_node = Node(start, heuristic=heuristic_func(start, goal))
    open_list.put(start_node)

    while not open_list.empty():
        current_node = open_list.get()
        closed_list.add(tuple(map(tuple, current_node.state))) # add the current node to the explored list

        if np.array_equal(current_node.state, goal): # check if the goal state is reached
            print(f"Goal state reached: {current_node.state}")
            return reconstruct_path(current_node)

        for action, next_state in get_successors(current_node.state): # get the successors of the current node
            if tuple(map(tuple, next_state)) in closed_list:
                continue

            heuristic = heuristic_func(next_state, goal) # compute the heuristic value of the next node
            next_node = Node(next_state, parent=current_node, action=action, heuristic=heuristic) # create the next node
            open_list.put(next_node) # put the next node in the open list

    return None

def get_successors(state):
    successors = []
    for action in available_actions(state):
        new_state = do_action(state, action)
        successors.append((action, new_state)) # collect the next possible states

    return successors
    

def reconstruct_path(node):
    path = []
    while node.parent is not None:
        path.append(node.action)
        node = node.parent
    path.reverse()
    return path



In [ ]:

path = greedy_best_first_search(state, goal, h)
print(f"Path found: {path}")